# Toronto Neighborhood Exploration

## Part1 : Web Scraping from Wikipedia

### I used requests to get the html file form the wikipedia page and then tried to find any table contained in that html

In [91]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_page = requests.get(url)
soup = BeautifulSoup(wiki_page.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Drop all `Postal Code` that were `Not assigned`

In [71]:
df_drop = df[df.Borough != 'Not assigned']
df_drop

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [72]:
df_drop.shape

(103, 3)

## Part2 : Assign Latitudes and Longitudes

In [73]:
ll = pd.read_csv('Geospatial_Coordinates.csv')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [94]:
df_combine = df_drop.merge(ll,on='Postal Code')
df_combine.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part3 : Clustering Neighborhoods

### Display neighborhood on map with blue marker

In [97]:
import folium
latitude = 43.651070
longitude = -79.347015
toronto_map = folium.Map(location=[latitude,longitude],zoom_start=12)

In [76]:
for lat, lng, borough, neighborhood in zip(df_combine['Latitude'], df_combine['Longitude'], df_combine['Borough'], df_combine['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
toronto_map

### Get nearby venues for each neighborhood (limit to 10 venues per area in the return list)

In [77]:
client_id = "WGENET1SWCZBUQYDFZDF5DGW5AIIE1THJCXUCQ1DW4PTHJNH"
client_secret = "CHCVAO54RWBGKU0IXZGZMVZCEG4DBCAPLOGNEWQNNW3V2MQJ"
version = "20180605"

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            10)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
venues = getNearbyVenues(names=df_combine['Neighborhood'],latitudes=df_combine.Latitude,longitudes=df_combine.Longitude)

### Venue list overview

In [80]:
venues.shape

(428, 7)

- In total `428` datapoints returned

In [81]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### Which Neighborhood had most venues returned?

In [82]:
venues.groupby('Neighborhood').Venue.count().sort_values(ascending=False)

Neighborhood
Downsview                                        16
Don Mills                                         9
Leaside                                           5
Christie                                          5
Commerce Court, Victoria Hotel                    5
                                                 ..
The Kingsway, Montgomery Road, Old Mill North     2
Weston                                            2
York Mills, Silver Hills                          2
Rouge Hill, Port Union, Highland Creek            1
Humber Summit                                     1
Name: Venue, Length: 95, dtype: int64

- Seems like `Downsview` had 16 venues returned

### How many unique venue categories were in the dataset?

In [83]:
venues['Venue Category'].nunique()

150

In [84]:
venues['Venue Category'].value_counts()

Coffee Shop            28
Park                   23
Restaurant             16
Pizza Place            15
Café                   15
                       ..
Fried Chicken Joint     1
Health Food Store       1
Airport Terminal        1
Cuban Restaurant        1
Fish Market             1
Name: Venue Category, Length: 150, dtype: int64

- `150` unique venue categories
- `Coffee Shop` was appeared to be the most frequent venue category

### Most common venues for each neighborhood?

In [88]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues['Neighborhood']

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Dog Run,Farmers Market,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pub,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Deli / Bodega,Bridal Shop,Restaurant,Yoga Studio,Distribution Center,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
3,Bayview Village,Bank,Café,Japanese Restaurant,Chinese Restaurant,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
4,"Bedford Park, Lawrence Manor East",Thai Restaurant,Café,Italian Restaurant,Indian Restaurant,Restaurant,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


### Running k-means algorithm to cluster neighborhoods

In [93]:
from sklearn.cluster import KMeans
k = 5
toronto_cluster = toronto_grouped.drop('Neighborhood',axis=1)

model = KMeans(n_clusters=k,random_state=0)
model.fit(toronto_cluster)
model.labels_[0:10]

array([2, 1, 4, 3, 3, 3, 3, 4, 2, 2])

In [96]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', model.labels_)

toronto_merged = df_combine
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Convenience Store,Park,Food & Drink Shop,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,French Restaurant,Curling Ice,Cuban Restaurant,Dance Studio,Creperie,Deli / Bodega
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Breakfast Spot,Coffee Shop,Bakery,Spa,Distribution Center,Yoga Studio,Dog Run,Farmers Market,Electronics Store,Eastern European Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Boutique,Vietnamese Restaurant,Coffee Shop,Clothing Store,Furniture / Home Store,Yoga Studio,Distribution Center,Electronics Store,Eastern European Restaurant,Drugstore
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Creperie,Italian Restaurant,Park,Coffee Shop,Distribution Center,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0.0,River,Park,Yoga Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2.0,Bubble Tea Shop,Dance Studio,Gastropub,Theme Restaurant,Mexican Restaurant,Yoga Studio,Distribution Center,Eastern European Restaurant,Drugstore,Donut Shop
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,2.0,Brewery,Farmers Market,Comic Shop,Burrito Place,Pizza Place,General Entertainment,Diner,Eastern European Restaurant,Drugstore,Donut Shop
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2.0,Construction & Landscaping,Baseball Field,Yoga Studio,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [110]:
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

In [113]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each cluster

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Convenience Store,Park,Food & Drink Shop,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0.0,Park,Spa,Women's Store,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,0.0,Convenience Store,Park,Coffee Shop,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0,Park,Bus Line,Swim School,Yoga Studio,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
64,M9N,York,Weston,43.706876,-79.518188,0.0,Convenience Store,Park,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Diner
66,M2P,North York,York Mills West,43.752758,-79.400049,0.0,Park,Convenience Store,Bank,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0.0,Tennis Court,Playground,Park,Summer Camp,Yoga Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,0.0,Playground,Park,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Dessert Shop
91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0.0,Park,Playground,Trail,Yoga Studio,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0.0,River,Park,Yoga Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


- Cluster 0 grouped neighborhoods with common venues like `Park`,`Convenience Store`

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,French Restaurant,Curling Ice,Cuban Restaurant,Dance Studio,Creperie,Deli / Bodega
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Bank,Gastropub,Pharmacy,Gym / Fitness Center,American Restaurant,Distribution Center,Farmers Market,Electronics Store,Eastern European Restaurant
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,1.0,Pizza Place,Cosmetics Shop,Liquor Store,Beer Store,Coffee Shop,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
50,M9L,North York,Humber Summit,43.756303,-79.565963,1.0,Pizza Place,Discount Store,Farmers Market,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Yoga Studio
56,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,1.0,Fast Food Restaurant,Coffee Shop,Sandwich Place,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
70,M9P,Etobicoke,Westmount,43.696319,-79.532242,1.0,Pizza Place,Coffee Shop,Sandwich Place,Chinese Restaurant,Curling Ice,Dance Studio,Cuban Restaurant,Deli / Bodega,Department Store,Electronics Store
72,M2R,North York,"Willowdale, Willowdale West",43.782736,-79.442259,1.0,Pizza Place,Home Service,Pharmacy,Coffee Shop,Bank,Dance Studio,Curling Ice,Deli / Bodega,Cuban Restaurant,Department Store
76,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,1.0,Hotel,Burrito Place,Coffee Shop,Sandwich Place,Yoga Studio,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1.0,Pizza Place,Mobile Phone Shop,Park,Sandwich Place,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1.0,Pizza Place,Gym,Coffee Shop,Pub,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


- Cluster 1 grouped neighborhoods with common venues like `Pizza Place`

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Print Shop,Diner,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Comic Shop,Plaza,Clothing Store,Theater,Tea Room,Yoga Studio,Diner,Eastern European Restaurant,Drugstore,Donut Shop
10,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Pizza Place,Japanese Restaurant,Asian Restaurant,Pub,Sushi Restaurant,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Yoga Studio,Distribution Center,Farmers Market,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
14,M4C,East York,Woodbine Heights,43.695344,-79.318389,2.0,Pharmacy,Curling Ice,Dance Studio,Beer Store,Skating Rink,Distribution Center,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
16,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,2.0,Field,Hockey Arena,Trail,Dog Run,Diner,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Distribution Center
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Bank,Rental Car Location,Medical Center,Mexican Restaurant,Electronics Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dance Studio
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2.0,Health Food Store,Trail,Pub,Yoga Studio,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Thai Restaurant,Department Store,Dessert Shop,Diner,Deli / Bodega,Discount Store
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,2.0,Golf Course,Pool,Mediterranean Restaurant,Athletics & Sports,Dog Run,Yoga Studio,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop


- Cluster 2 grouped neighborhoods with common venues like `Donut Shop`,`Drugstore`,`Dog Run`

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M3B,North York,Don Mills,43.745906,-79.352188,3.0,Gym,Restaurant,Gym / Fitness Center,Café,Caribbean Restaurant,Discount Store,Clothing Store,Japanese Restaurant,Italian Restaurant,Eastern European Restaurant
13,M3C,North York,Don Mills,43.725900,-79.340923,3.0,Gym,Restaurant,Gym / Fitness Center,Café,Caribbean Restaurant,Discount Store,Clothing Store,Japanese Restaurant,Italian Restaurant,Eastern European Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3.0,Liquor Store,Concert Hall,Vegetarian / Vegan Restaurant,Museum,Restaurant,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
23,M4G,East York,Leaside,43.709060,-79.363452,3.0,Sporting Goods Shop,Restaurant,Gym,Sports Bar,Fish & Chips Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
29,M4H,East York,Thorncliffe Park,43.705369,-79.349372,3.0,Yoga Studio,Indian Restaurant,Pharmacy,Gym,Warehouse Store,Golf Course,Dessert Shop,Drugstore,Donut Shop,Dog Run
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3.0,Concert Hall,Plaza,Vegetarian / Vegan Restaurant,Café,Restaurant,Diner,Drugstore,Donut Shop,Dog Run,Distribution Center
39,M2K,North York,Bayview Village,43.786947,-79.385975,3.0,Bank,Café,Japanese Restaurant,Chinese Restaurant,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,3.0,Gym,Restaurant,Gym / Fitness Center,Café,Bakery,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3.0,Movie Theater,American Restaurant,Motel,Yoga Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,3.0,Thai Restaurant,Café,Italian Restaurant,Indian Restaurant,Restaurant,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


- Cluster 3 grouped neighborhoods with common venues like `Gym`,`Cafe`,`Restaurant`

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Breakfast Spot,Coffee Shop,Bakery,Spa,Distribution Center,Yoga Studio,Dog Run,Farmers Market,Electronics Store,Eastern European Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Boutique,Vietnamese Restaurant,Coffee Shop,Clothing Store,Furniture / Home Store,Yoga Studio,Distribution Center,Electronics Store,Eastern European Restaurant,Drugstore
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Creperie,Italian Restaurant,Park,Coffee Shop,Distribution Center,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4.0,Coffee Shop,Creperie,Japanese Restaurant,Restaurant,Dance Studio,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Farmers Market
22,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4.0,Coffee Shop,Spa,Middle Eastern Restaurant,Yoga Studio,Discount Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4.0,Café,Italian Restaurant,Grocery Store,Coffee Shop,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,4.0,Coffee Shop,Deli / Bodega,Bridal Shop,Restaurant,Yoga Studio,Distribution Center,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,4.0,Discount Store,Chinese Restaurant,Coffee Shop,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4.0,Gym,Restaurant,Coffee Shop,Café,Bakery,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


- Cluster 4 grouped neighborhoods with common venues like `Coffee Shop`